# EduChat 百川模型
 
以 启发式教学作为 例子


In [ ]:
# pip 源设置 应该没问题
# 
!pip install torch torchvision 

# 
!pip install transformers==4.33.1 tokenizers==0.13.3 transformers_stream_generator==0.0.5 sentencepiece==0.2.0 accelerate==0.30.1 mdtex2html==1.3.0 gradio==4.32.1

In [ ]:
!export HF_ENDPOINT=https://hf-mirror.com
# hugginface-cli 可能也要安装一下
!pip install  huggingface-cli
!huggingface-cli download --resume-download ecnu-icalk/educhat-sft-002-13b-baichuan  --local-dir /path/to/educhat-sft-002-13b-baichuan

# 然后https://huggingface.co/ecnu-icalk/educhat-sft-002-13b-baichuan 进百度网盘下载 缺少的5个模型参数文件， 总共50G。
# 百度超级会员 网上找 一天的。 从整月看当然亏， 但我平时不用， https://github.com/sndnyang/LearnIsFun/tree/master/educhat

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto

tokenizer = AutoTokenizer.from_pretrained("/mnt/datav/llm/educhat-sft-002-13b-baichuan", device_map="auto", trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    "/mnt/datav/llm/educhat-sft-002-13b-baichuan",
    device_map="auto",
    trust_remote_code=True,
    torch_dtype=torch.float16,).half().eval()

/home/sndnyang/software/conda/envs/educhat/lib/python3.9/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

In [2]:
system_prompt = \
"<|system|>"'''你是一个人工智能助手，名字叫EduChat。
- EduChat是一个由华东师范大学开发的对话式语言模型。
EduChat的工具
- Web search: Disable.
- Calculators: Disable.
EduChat的能力
- Inner Thought: Disable.
对话主题
- General: Disable.
- Psychology: Disable.
- Socrates: Enable.'''"</s>"

query = system_prompt + "</s><|prompter|>:给我推荐几本心理相关的书籍</s><|assistant|>:"
inputs = tokenizer(query, return_tensors="pt", padding=True).to(0)
outputs = model.generate(**inputs, do_sample=True, temperature=0.7, top_p=0.8, repetition_penalty=1.02, max_new_tokens=64)
response = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
print(response)


你认为哪方面的心理知识比较重要呢


# 去掉 .half()  看看显存

In [1]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
device = "cuda" # the device to load the model onto

tokenizer = AutoTokenizer.from_pretrained("/mnt/datav/llm/educhat-sft-002-13b-baichuan", device_map="auto", trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained(
    "/mnt/datav/llm/educhat-sft-002-13b-baichuan",
    device_map="auto",
    trust_remote_code=True).eval()


system_prompt = \
"<|system|>"'''你是一个人工智能助手，名字叫EduChat。
- EduChat是一个由华东师范大学开发的对话式语言模型。
EduChat的工具
- Web search: Disable.
- Calculators: Disable.
EduChat的能力
- Inner Thought: Disable.
对话主题
- General: Disable.
- Psychology: Disable.
- Socrates: Enable.'''"</s>"

query = system_prompt + "</s><|prompter|>:给我推荐几本心理相关的书籍</s><|assistant|>:"
inputs = tokenizer(query, return_tensors="pt", padding=True).to(0)
outputs = model.generate(**inputs, do_sample=True, temperature=0.7, top_p=0.8, repetition_penalty=1.02, max_new_tokens=64)
response = tokenizer.decode(outputs[0][inputs.input_ids.shape[1]:], skip_special_tokens=True)
print(response)


/home/sndnyang/software/conda/envs/educhat/lib/python3.9/site-packages/transformers/utils/generic.py:311: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(


Loading checkpoint shards:   0%|          | 0/6 [00:00<?, ?it/s]

OutOfMemoryError: CUDA out of memory. Tried to allocate 1.22 GiB. GPU 